# 端到端测试：模拟拉比振荡

本 Notebook 将使用 `QuantumSim` 库的所有核心模块，进行一次完整的端到端仿真测试。

**目标**:
1.  初始化一个 `DuffingOscillatorModel`。
2.  创建一个微波驱动序列 (`MicrowaveSequence`)。
3.  使用 `Simulator` 在 RWA 模式下进行仿真。
4.  绘制布居数和布洛赫矢量随时间的演化，并与理论结果对比，以验证整个框架的正确性。

## 1.基本参数

In [ ]:
%load_ext autoreload
%autoreload 2


import numpy as np
import qutip
from qutip import basis, sigmax, sigmay, sigmaz
import matplotlib.pyplot as plt

# 导入我们自己库中的所有组件
from quantum_sim import (
    DuffingOscillatorModel,
    MicrowaveSequence,
    Simulator,
    CosineEnvelope,
    plotting
)


In [ ]:
# --- 1. 物理系统参数 ---
D_LEVELS = 15
OMEGA_Q_GHZ = 5.3
ANH_GHZ = -0.212

# --- 2. 脉冲参数 ---
PULSE_DURATION = 20 # ns, 设置为一个典型的 pi 脉冲时长
# 对于余弦包络 Omega(t) = (A/2)*(1-cos(2*pi*t/tau))
# 其脉冲面积为 A * tau / 2。
# 为了实现一个 pi 脉冲 (面积=pi)，我们需要设置峰值振幅 A = 2*pi/tau
PULSE_AMP_RAD_NS = 2 * np.pi / PULSE_DURATION

# --- 3. 仿真时间轴 ---
T_LIST = np.linspace(0, PULSE_DURATION, 1001)

T_LIST_FINE = np.linspace(0, PULSE_DURATION, 4001)


# --- 4. 初始状态 ---
PSI0 = basis(D_LEVELS, 0)

## 2.建模仿真

In [ ]:
# 1. 创建物理模型实例 (使用 rad/ns 单位)
model = DuffingOscillatorModel(
    d=D_LEVELS,
    omega_q=OMEGA_Q_GHZ * 2 * np.pi,
    anh=ANH_GHZ * 2 * np.pi
)

# 2. 创建一个余弦包络的微波驱动序列
drive_envelope = CosineEnvelope(duration=PULSE_DURATION, amp=PULSE_AMP_RAD_NS)
mw_sequence = MicrowaveSequence(envelope=drive_envelope, carrier_freq=model.omega_q)

# 3. 创建用户信号字典
sequences_dict = {
    'XY_drive': mw_sequence
}

# 4. 创建仿真器实例
sim = Simulator(model=model)

print("所有仿真组件已成功创建 (使用 CosineEnvelope)。")

### 2.1 基于 RWA 的模型的仿真

In [ ]:
# 定义需要计算期望值的算符
sx_op = basis(D_LEVELS, 0) * basis(D_LEVELS, 1).dag() + basis(D_LEVELS, 1) * basis(D_LEVELS, 0).dag()
sy_op = -1j * (basis(D_LEVELS, 0) * basis(D_LEVELS, 1).dag() - basis(D_LEVELS, 1) * basis(D_LEVELS, 0).dag())
sz_op = basis(D_LEVELS, 0) * basis(D_LEVELS, 0).dag() - basis(D_LEVELS, 1) * basis(D_LEVELS, 1).dag()
e_ops = [
    sx_op,
    sy_op,
    sz_op,
    basis(D_LEVELS, 0) * basis(D_LEVELS, 0).dag(), # P(|0>)
    basis(D_LEVELS, 1) * basis(D_LEVELS, 1).dag(), # P(|1>)
    basis(D_LEVELS, 2) * basis(D_LEVELS, 2).dag()  # P(|2>)
]

# 运行仿真！
print("正在运行 RWA 仿真...")
result_rwa = sim.run(
    psi0=PSI0,
    t_list=T_LIST,
    sequences=sequences_dict,
    show_waveforms=True,
    use_rwa=True,
    e_ops=e_ops
)
print("...仿真完成。")

sx_value_rwa = result_rwa.expect[0]
sy_value_rwa = result_rwa.expect[1]
sz_value_rwa = result_rwa.expect[2]

pop0_rwa = result_rwa.expect[3]
pop1_rwa = result_rwa.expect[4]
pop2_rwa = result_rwa.expect[5]

fig, axes = plt.subplots(2, 1, figsize=(10,6))
axes[0].plot(T_LIST, pop0_rwa, label='P(|0>)')
axes[0].plot(T_LIST, pop1_rwa, label='P(|1>)')
axes[0].plot(T_LIST, pop2_rwa, label='P(|2>)')
axes[0].set_xlabel('time (ns)')
axes[0].set_ylabel('probability')
axes[0].legend()

axes[1].plot(T_LIST, sx_value_rwa, label='<Sx>')
axes[1].plot(T_LIST, sy_value_rwa, label='<Sy>')
axes[1].plot(T_LIST, sz_value_rwa, label='<Sz>')
axes[1].set_xlabel('time (ns)')
axes[1].set_ylabel('expectation value')
axes[1].legend()

plt.tight_layout()
plt.show()


### 2.2 基于实验室坐标系的建模仿真

In [ ]:
# 定义需要计算期望值的算符
sx_op = basis(D_LEVELS, 0) * basis(D_LEVELS, 1).dag() + basis(D_LEVELS, 1) * basis(D_LEVELS, 0).dag()
sy_op = -1j * (basis(D_LEVELS, 0) * basis(D_LEVELS, 1).dag() - basis(D_LEVELS, 1) * basis(D_LEVELS, 0).dag())
sz_op = basis(D_LEVELS, 0) * basis(D_LEVELS, 0).dag() - basis(D_LEVELS, 1) * basis(D_LEVELS, 1).dag()
e_ops = [
    sx_op,
    sy_op,
    sz_op,
    basis(D_LEVELS, 0) * basis(D_LEVELS, 0).dag(), # P(|0>)
    basis(D_LEVELS, 1) * basis(D_LEVELS, 1).dag(), # P(|1>)
    basis(D_LEVELS, 2) * basis(D_LEVELS, 2).dag()  # P(|2>)
]



# 运行仿真！
print("正在运行 RWA 仿真...")
result_lab = sim.run(
    psi0=PSI0,
    t_list=T_LIST_FINE,
    sequences=sequences_dict,
    use_rwa=False,
    e_ops=e_ops
)
print("...仿真完成。")

sx_value = result_lab.expect[0]
sy_value = result_lab.expect[1]
sz_value = result_lab.expect[2]

pop0 = result_lab.expect[3]
pop1 = result_lab.expect[4]
pop2 = result_lab.expect[5]

fig, axes = plt.subplots(2, 1, figsize=(10,6))
axes[0].plot(T_LIST_FINE, pop0, label='P(|0>)')
axes[0].plot(T_LIST_FINE, pop1, label='P(|1>)')
axes[0].plot(T_LIST_FINE, pop2, label='P(|2>)')
axes[0].set_xlabel('time (ns)')
axes[0].set_ylabel('probability')
axes[0].legend()

axes[1].plot(T_LIST_FINE, sx_value, label='<Sx>')
axes[1].plot(T_LIST_FINE, sy_value, label='<Sy>')
axes[1].plot(T_LIST_FINE, sz_value, label='<Sz>')
axes[1].set_xlabel('time (ns)')
axes[1].set_ylabel('expectation value')
axes[1].legend()

plt.tight_layout()
plt.show()


### 2.3 将实验室坐标系的结果转换到旋转坐标系中

In [ ]:
from qutip import expect
from tqdm.notebook import tqdm # 导入tqdm以显示进度条

# 1. 从模型中获取 Lab Frame 的自由哈密顿量
h_drift_lab = model.get_drift_hamiltonian(use_rwa=False)

# 2. 准备存储变换后结果的列表
lab_states = result_lab.states
t_points = result_lab.times
exp_vals_rotating = [[] for _ in e_ops] # 每个算符一个空列表

print("正在将 Lab Frame 结果变换到 Rotating Frame...")
# 3. 遍历每一个时间点和对应的 lab frame 状态
for t, psi_lab in tqdm(zip(t_points, lab_states), total=len(t_points)):
    # 4. 计算演化算符 U_0†(t) = exp(i * H_0 * t)
    U0_dag = (1j * h_drift_lab * t).expm()
    
    # 5. 应用变换
    psi_rot = U0_dag * psi_lab
    
    # 6. 计算并存储变换后状态的期望值
    for i, op in enumerate(e_ops):
        exp_vals_rotating[i].append(expect(op, psi_rot))

print("...变换完成。")

## 3.结果可视化

### 3.1 多种方法的结果对比

In [ ]:
# 为了绘图清晰，我们将 RWA 的结果也插值到高分辨率的时间轴上
result_rwa_interp = [np.interp(T_LIST_FINE, T_LIST, result_rwa.expect[i]) for i in range(len(e_ops))]

# --- 绘图 ---
fig, axes = plt.subplots(4, 1, figsize=(14, 10), sharex=True)
fig.suptitle("RWA vs. Lab Frame vs. Rotating Frame", fontsize=18)

# --- 子图1: 布居数对比 ---
# RWA 结果
axes[0].plot(T_LIST, result_rwa.expect[3], 'C0-', label=r"$P(|0\rangle)$ - RWA")
axes[0].plot(T_LIST, result_rwa.expect[4], 'C2-', label=r"$P(|1\rangle)$ - RWA")
axes[0].plot(T_LIST, result_rwa.expect[5], 'C4-', label=r"$P(|2\rangle)$ - RWA")
# Lab Frame 变换后的结果
axes[0].plot(t_points, exp_vals_rotating[3], 'C0:', lw=3, label=r"$P(|0\rangle)$ - Rotating")
axes[0].plot(t_points, exp_vals_rotating[4], 'C2:', lw=3, label=r"$P(|1\rangle)$ - Rotating")
axes[0].plot(t_points, exp_vals_rotating[5], 'C4:', lw=3, label=r"$P(|2\rangle)$ - Rotating")

axes[0].set_title("Populations")
axes[0].set_ylabel("Probability")
axes[0].legend()
axes[0].grid(True)

# --- 子图2: sigma_x期望值对比 ---

axes[1].plot(t_points, result_lab.expect[0], 'C1--', alpha=0.7, label=r"$\langle\sigma_x\rangle$ - Lab Frame (Original)")
axes[1].plot(T_LIST, result_rwa.expect[0], 'C1-', lw=2, label=r"$\langle\sigma_x\rangle$ - RWA")
axes[1].plot(t_points, exp_vals_rotating[0], 'm:', lw=3, label=r"$\langle\sigma_x\rangle$ - Rotating")

axes[1].set_title("Expectation Values ($\sigma_x$)")
axes[1].set_xlabel("Time (ns)")
axes[1].set_ylabel("Expectation Value")
axes[1].legend()
axes[1].grid(True)

# --- 子图3: sigma_y期望值对比 ---
axes[2].plot(t_points, result_lab.expect[1], 'C2--', alpha=0.7, label=r"$\langle\sigma_y\rangle$ - Lab Frame (Original)")
axes[2].plot(T_LIST, result_rwa.expect[1], 'C2-', lw=2, label=r"$\langle\sigma_y\rangle$ - RWA")
axes[2].plot(t_points, exp_vals_rotating[1], 'm:', lw=3, label=r"$\langle\sigma_y\rangle$ - Rotating")

axes[2].set_title("Expectation Values ($\sigma_y$)")
axes[2].set_xlabel("Time (ns)")
axes[2].set_ylabel("Expectation Value")
axes[2].legend()
axes[2].grid(True)

# --- 子图4: sigma_z期望值对比 ---
axes[3].plot(t_points, result_lab.expect[2], 'C3--', alpha=0.7, label=r"$\langle\sigma_z\rangle$ - Lab Frame (Original)")
# RWA 结果 (慢变)
axes[3].plot(T_LIST, result_rwa.expect[2], 'C3-', lw=2, label=r"$\langle\sigma_z\rangle$ - RWA")
# Lab Frame 变换后的结果 (慢变)
axes[3].plot(t_points, exp_vals_rotating[2], 'm:', lw=3, label=r"$\langle\sigma_z\rangle$ - Rotating")

axes[3].set_title("Expectation Values ($\sigma_z$)")
axes[3].set_xlabel("Time (ns)")
axes[3].set_ylabel("Expectation Value")
axes[3].legend()
axes[3].grid(True)



plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

### 3.2 Bloch球的可视化

#### 3维可视化

In [ ]:
from qutip import Bloch


exp_sz = exp_vals_rotating[2]
exp_sx = exp_vals_rotating[0]
exp_sy = exp_vals_rotating[1]

# 将坐标数据打包成 qutip.Bloch 需要的格式
points = [exp_sx, exp_sy, exp_sz]

# --- 2. 创建并绘制布洛赫球 ---
# 实例化一个 Bloch 球对象
b = Bloch()

# 设置标题 (这是正确的方式)
b.title = "Qubit State Trajectory on the Bloch Sphere (RWA)"

b.add_points(points, meth='l')

# --- 關鍵步驟：為起點和終點設置樣式 ---
# 根據源碼，我們通過設置 Bloch 對象的屬性來定義後續添加的“點”的樣式。
# 我們接下來要添加2個點，所以顏色列表裡準備2個顏色。
b.point_color = ['green', 'red']
# 為了確保標記是圓形，也可以設置 point_marker 屬性
b.point_marker = ['o', 'o']
# 也可以調整點的大小
b.point_size = [30, 40]

# 添加起點標記
# 這次 add_points 是添加“點”，會使用 b.point_color[0] 和 b.point_marker[0]
b.add_points([points[0][0], points[1][0], points[2][0]])

# 添加終點標記
# 這次 add_points 是添加“點”，會使用 b.point_color[1] 和 b.point_marker[1]
b.add_points([points[0][-1], points[1][-1], points[2][-1]])

# 渲染並顯示
b.show()

#### 三视图可视化

In [ ]:
n_steps = len(exp_sz)
cmap = plt.get_cmap('coolwarm') 
colors = [cmap(i/n_steps) for i in range(n_steps)]

# --- 3. 創建三視圖 ---
# 創建一個 1x3 的子圖佈局，figsize 可以根據你的屏幕調整
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4.5))
fig.suptitle('2D Projections of Bloch Sphere Trajectory (Three-View)', fontsize=16)

# --- 子圖 1: XY 投影 (俯視圖) ---
# 繪製布洛赫球在該平面的輪廓 (單位圓)
circle = plt.Circle((0, 0), 1, color='gray', fill=False, linestyle='--')
ax1.add_artist(circle)
# 使用 scatter 繪製帶有顏色梯度的軌跡
ax1.scatter(exp_sx, exp_sy, c=colors, s=10)
# 標記起點和終點
ax1.scatter(exp_sx[0], exp_sy[0], color='green', s=50, label='Start', zorder=5)
ax1.scatter(exp_sx[-1], exp_sy[-1], color='black', marker='x', s=50, label='End', zorder=5)
# 設置圖形屬性
ax1.set_title('XY Projection (Top View)')
ax1.set_xlabel(r'$\sigma_x$')
ax1.set_ylabel(r'$\sigma_y$')
ax1.set_aspect('equal', adjustable='box') # 關鍵：確保圓形不會被壓扁
ax1.set_xlim([-1.1, 1.1])
ax1.set_ylim([-1.1, 1.1])
ax1.grid(True)
ax1.legend()

# --- 子圖 2: XZ 投影 (主視圖) ---
circle = plt.Circle((0, 0), 1, color='gray', fill=False, linestyle='--')
ax2.add_artist(circle)
ax2.scatter(exp_sx, exp_sz, c=colors, s=10)
ax2.scatter(exp_sx[0], exp_sz[0], color='green', s=50, label='Start', zorder=5)
ax2.scatter(exp_sx[-1], exp_sz[-1], color='black', marker='x', s=50, label='End', zorder=5)
ax2.set_title('XZ Projection (Front View)')
ax2.set_xlabel(r'$\sigma_x$')
ax2.set_ylabel(r'$\sigma_z$')
ax2.set_aspect('equal', adjustable='box')
ax2.set_xlim([-1.1, 1.1])
ax2.set_ylim([-1.1, 1.1])
ax2.grid(True)
ax2.legend()

# --- 子圖 3: YZ 投影 (側視圖) ---
circle = plt.Circle((0, 0), 1, color='gray', fill=False, linestyle='--')
ax3.add_artist(circle)
ax3.scatter(exp_sy, exp_sz, c=colors, s=10)
ax3.scatter(exp_sy[0], exp_sz[0], color='green', s=50, label='Start', zorder=5)
ax3.scatter(exp_sy[-1], exp_sz[-1], color='black', marker='x', s=50, label='End', zorder=5)
ax3.set_title('YZ Projection (Side View)')
ax3.set_xlabel(r'$\sigma_y$')
ax3.set_ylabel(r'$\sigma_z$')
ax3.set_aspect('equal', adjustable='box')
ax3.set_xlim([-1.1, 1.1])
ax3.set_ylim([-1.1, 1.1])
ax3.grid(True)
ax3.legend()

# 調整佈局並顯示圖形
plt.tight_layout(rect=[0, 0, 1, 0.96]) # 為 suptitle 留出空間
plt.show()

## 4. DRAG 效果测试



In [ ]:
from quantum_sim import DRAGCorrector


D_LEVELS = 10
PULSE_DURATION_DRAG = 10
PULSE_AMP = np.pi / PULSE_DURATION_DRAG
# PULSE_AMP = 0.4

T_LIST_DRAG = np.linspace(0, PULSE_DURATION_DRAG * 1, 4001) # 专门为了研究DRAG重新选择了时间

model = DuffingOscillatorModel(
    d=D_LEVELS,
    omega_q=OMEGA_Q_GHZ * 2 * np.pi,
    anh=ANH_GHZ * 2 * np.pi
)
PSI0 = basis(D_LEVELS, 0)

sx_op = basis(D_LEVELS, 0) * basis(D_LEVELS, 1).dag() + basis(D_LEVELS, 1) * basis(D_LEVELS, 0).dag()
sy_op = -1j * (basis(D_LEVELS, 0) * basis(D_LEVELS, 1).dag() - basis(D_LEVELS, 1) * basis(D_LEVELS, 0).dag())
sz_op = basis(D_LEVELS, 0) * basis(D_LEVELS, 0).dag() - basis(D_LEVELS, 1) * basis(D_LEVELS, 1).dag()
e_ops = [
    sx_op,
    sy_op,
    sz_op,
    basis(D_LEVELS, 0) * basis(D_LEVELS, 0).dag(), # P(|0>)
    basis(D_LEVELS, 1) * basis(D_LEVELS, 1).dag(), # P(|1>)
    basis(D_LEVELS, 2) * basis(D_LEVELS, 2).dag()  # P(|2>)
]

# 1. 创建基础的余弦包络和微波序列 (无 DRAG)
cos_env_test = CosineEnvelope(duration=PULSE_DURATION_DRAG, amp=PULSE_AMP)
mw_seq_no_drag = MicrowaveSequence(envelope=cos_env_test, carrier_freq=model.omega_q)

# 2. 创建 DRAG 修正器并应用
# 我们使用 alpha=1.0 以最大化对泄漏的抑制
drag_corrector = DRAGCorrector(anh=model.anh, alpha=1.12)
mw_seq_with_drag = drag_corrector.apply(mw_seq_no_drag)

# 3. 为 sim.run 准备信号字典
sequences_no_drag = {'XY_drive': mw_seq_no_drag}
sequences_with_drag = {'XY_drive': mw_seq_with_drag}

sim = Simulator(model=model)

# --- 运行两次仿真 ---
print("正在运行无 DRAG 的仿真...")
result_no_drag = sim.run(
    psi0=PSI0,
    t_list=T_LIST_DRAG, 
    sequences=sequences_no_drag,
    use_rwa=True,
    e_ops=e_ops
)
print("...完成。")

print("正在运行有 DRAG 的仿真...")
result_with_drag = sim.run(
    psi0=PSI0,
    t_list=T_LIST_DRAG, 
    sequences=sequences_with_drag,
    show_waveforms=True,
    use_rwa=True,
    e_ops=e_ops
)
print("...完成。")


# --- 结果可视化对比 ---
fig, axes = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
fig.suptitle(r"DRAG Correction Effect on Cosine $\pi$-Pulse (t=20ns)", fontsize=18)

# --- 子图1: |0> 和 |1> 态布居数 ---
# 提取布居数数据
pop1_no_drag = result_no_drag.expect[4]
pop1_with_drag = result_with_drag.expect[4]

axes[0].plot(T_LIST_DRAG, pop1_no_drag, '--', alpha=0.8, label=r"$P(|1\rangle)$ - No DRAG")
axes[0].plot(T_LIST_DRAG, pop1_with_drag, '-', lw=2, label=r"$P(|1\rangle)$ - With DRAG")
axes[0].set_title("Population in Computational Subspace")
axes[0].set_ylabel("Population")
axes[0].legend()
axes[0].grid(True)
axes[0].set_ylim(-0.05, 1.05)


# --- 子图2: |2> 态泄漏布居数 (关键对比) ---
# 提取泄漏数据
leakage_no_drag = 1 - (result_no_drag.expect[3] + result_no_drag.expect[4])
leakage_with_drag = 1 - (result_with_drag.expect[3] + result_with_drag.expect[4])

axes[1].plot(T_LIST_DRAG, leakage_no_drag, 'r--', alpha=0.8, label=r"$P(|2\rangle)$ - No DRAG (Leakage)")
axes[1].plot(T_LIST_DRAG, leakage_with_drag, 'r-', lw=2, label=r"$P(|2\rangle)$ - With DRAG (Leakage Suppressed)")
axes[1].set_title("Population in Leakage State |2> (Log Scale)")
axes[1].set_xlabel("Time (ns)")
axes[1].set_ylabel("Leakage Population")
axes[1].set_yscale('log') # 使用对数坐标轴以清晰显示微小的泄漏
axes[1].legend()
axes[1].grid(True, which='both')


plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# 打印最终的泄漏值以作定量对比
print(f"最终时刻 (t={PULSE_DURATION_DRAG}ns):")
print(f"  - 无 DRAG 时的泄漏: {leakage_no_drag[-1]:.2e}")
print(f"  - 有 DRAG 时的泄漏: {leakage_with_drag[-1]:.2e}")

### 4.1 用扫描器扫描alpha系数

In [ ]:
# 在您的测试脚本中
import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, ptrace
from quantum_sim import DuffingOscillatorModel, CosineEnvelope, MicrowaveSequence, DRAGCorrector, Simulator
from quantum_sim.scanner import Scanner # 导入我们新的 Scanner
from qutip import expect, Result

# --- 1. 定义固定部分 (模型, 脉冲形状, 初始条件等) ---


# --- 2. 准备 Scanner ---
# 仿真中不变的参数
base_args = {
    'psi0': PSI0,
    't_list': T_LIST_DRAG,
    'use_rwa': True,
    'e_ops': e_ops,
}

sim = Simulator(model=model, 
                options={
                    'nsteps': 50000,
                    'atol': 1e-9,
                    'rtol': 1e-7,
                    'store_states': True
                }
            )
scanner = Scanner(simulator=sim, base_sim_args=base_args)

# --- 3. 定义扫描逻辑 (通过两个简单的函数) ---
# a) Setup 函数: 告诉 Scanner 如何根据 alpha 值构建 sequences 字典
def alpha_setup(alpha: float) -> dict:
    drag_corrector = DRAGCorrector(anh=model.anh, alpha=alpha)
    mw_seq_with_drag = drag_corrector.apply(mw_seq_no_drag)
    return {'sequences': {'XY_drive': mw_seq_with_drag}}

# b) Post-process 函数: 告诉 Scanner 如何从结果中提取最终泄漏率
def final_leakage_processor(result: Result) -> float:
    # 假设 e_ops[3] 和 e_ops[4] 分别是 P(|0>) 和 P(|1>)
    leakage_at_end = 1 - (result.expect[3][-1] + result.expect[4][-1])
    return leakage_at_end

# 定义扫描范围
alpha_values = np.linspace(0, 2.0, 51)

# --- 4. 执行扫描 ---
# 只需一行代码！
scanned_alphas, scanned_leakages = scanner.run(
    sweep_values=alpha_values,
    setup_function=alpha_setup,
    post_process_function=final_leakage_processor
)

# --- 5. 结果可视化 ---
plt.figure(figsize=(8, 6))
plt.plot(scanned_alphas, scanned_leakages, 'o-', label='Final Leakage vs. Alpha')
plt.xlabel("DRAG Coefficient (alpha)")
plt.ylabel("Final Leakage Population")
plt.yscale('log') # 同样建议使用对数坐标
plt.title("DRAG Leakage Suppression Sweep")
plt.grid(True, which='both')
plt.legend()
plt.show()

In [ ]:
scanned_alphas[np.where(scanned_leakages == np.min(scanned_leakages))[0][0]]

In [ ]:
from quantum_sim import DRAGCorrector

PULSE_DURATION = 10
PULSE_AMP_RAD_NS = 2 * np.pi / PULSE_DURATION
T_LIST = np.linspace(0, PULSE_DURATION, 1001)



# 2. 创建一个余弦包络的微波驱动序列
drive_envelope = CosineEnvelope(duration=PULSE_DURATION, amp=PULSE_AMP_RAD_NS)
mw_sequence = MicrowaveSequence(envelope=drive_envelope, carrier_freq=model.omega_q)

drag_corrector = DRAGCorrector(alpha=0.0, anh=model.anh)
mw_sequence_drag = drag_corrector.apply(mw_sequence)

sequences_dict = {
    'XY_drive': mw_sequence_drag
}

# 定义需要计算期望值的算符
sx_op = basis(D_LEVELS, 0) * basis(D_LEVELS, 1).dag() + basis(D_LEVELS, 1) * basis(D_LEVELS, 0).dag()
sy_op = -1j * (basis(D_LEVELS, 0) * basis(D_LEVELS, 1).dag() - basis(D_LEVELS, 1) * basis(D_LEVELS, 0).dag())
sz_op = basis(D_LEVELS, 0) * basis(D_LEVELS, 0).dag() - basis(D_LEVELS, 1) * basis(D_LEVELS, 1).dag()
e_ops = [
    sx_op,
    sy_op,
    sz_op,
    basis(D_LEVELS, 0) * basis(D_LEVELS, 0).dag(), # P(|0>)
    basis(D_LEVELS, 1) * basis(D_LEVELS, 1).dag(), # P(|1>)
    basis(D_LEVELS, 2) * basis(D_LEVELS, 2).dag()  # P(|2>)
]

# 运行仿真！
result_rwa = sim.run(
    psi0=PSI0,
    t_list=T_LIST,
    sequences=sequences_dict,
    show_waveforms=True,
    use_rwa=True,
    e_ops=e_ops
)
print("...仿真完成。")

sx_value_rwa = result_rwa.expect[0]
sy_value_rwa = result_rwa.expect[1]
sz_value_rwa = result_rwa.expect[2]

pop0_rwa = result_rwa.expect[3]
pop1_rwa = result_rwa.expect[4]
pop2_rwa = result_rwa.expect[5]


fig, ax = plt.subplots(figsize=(6,4))
# ax.plot(T_LIST, pop0_rwa, label='P(|0>)')
# ax.plot(T_LIST, pop1_rwa, label='P(|1>)')
ax.plot(T_LIST, pop2_rwa, label='P(|2>)')
ax.set_xlabel('time (ns)')
ax.set_ylabel('probability')
ax.legend()
# ax.set_yscale('log')
ax.grid()

fig, axes = plt.subplots(2, 1, figsize=(10,6))
axes[0].plot(T_LIST, pop0_rwa, label='P(|0>)')
axes[0].plot(T_LIST, pop1_rwa, label='P(|1>)')
axes[0].plot(T_LIST, pop2_rwa, label='P(|2>)')
axes[0].set_xlabel('time (ns)')
axes[0].set_ylabel('probability')
axes[0].legend()

axes[1].plot(T_LIST, sx_value_rwa, label='<Sx>')
axes[1].plot(T_LIST, sy_value_rwa, label='<Sy>')
axes[1].plot(T_LIST, sz_value_rwa, label='<Sz>')
axes[1].set_xlabel('time (ns)')
axes[1].set_ylabel('expectation value')
axes[1].legend()

plt.tight_layout()
plt.show()
